1. Quyét dữ liệu inflation và tương đương

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By

In [2]:
# Tạo bảng
columns = ['MMYYYY','Bank_name', 'CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M','Onl_1Y','Onl_2Y']

Update_inst = pd.DataFrame(columns=columns)

url_chrome = r"D:\Chromedriver\chromedriver.exe"


In [3]:
MacroE_url = 'https://tradingeconomics.com/vietnam/inflation-cpi'

chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
MacroE = driver.get(MacroE_url)
driver.implicitly_wait(5)
Content = driver.page_source
soup = BeautifulSoup(Content, 'html.parser')

table = soup.find_all('tbody')[1]

# Tạo danh sách trống để lưu dữ liệu
data = []

# Lặp qua các hàng trong bảng
for row in table.find_all('tr'):
    # Lấy tất cả các cột trong mỗi hàng
    columns = row.find_all('td')
    # Lấy nội dung của các cột và lưu vào danh sách dữ liệu
    data.append([column.get_text() for column in columns])
Macro_E = pd.DataFrame(data)

In [54]:
Macro_E

,0,1,2,3,4
0,\n\n Inflat...,3.59,3.66,\n percent,Oct 2023
1,\n\n Consum...,113.38,113.28,\n points,Oct 2023
2,\n\n Core I...,3.43,4.49,\n percent,Oct 2023
3,\n\n GDP De...,171.54,165.17,\n points,Dec 2022
4,\n\n Export...,110.50,118.90,\n points,Dec 2022
5,\n\n Import...,107.80,126.70,\n points,Dec 2022
6,\n\n Food I...,2.81,2.87,\n percent,Oct 2023
7,\n\n CPI Tr...,110.76,112.46,\n points,Oct 2023
8,\n\n Inflat...,0.08,1.08,\n percent,Oct 2023
9,\n\n Produc...,97.51,98.48,\n points,Sep 2023


In [60]:
Update_inst.loc[0, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[0, 'Bank_name'] = 'Macro_Inflation_%'
Update_inst.loc[0, "CASA_int"] = Macro_E.iloc[0,1]

Update_inst.loc[1, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[1, 'Bank_name'] = 'Macro_ConsumerPriceIndex_%'
Update_inst.loc[1, "CASA_int"] = Macro_E.iloc[1,1]

Update_inst.loc[2, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[2, 'Bank_name'] = 'Macro_ProducerPrices_Point'
Update_inst.loc[2, "CASA_int"] = Macro_E.iloc[8,1]
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,31/10/2023,Macro_Inflation,3.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31/10/2023,Macro_ConsumerPriceIndex,113.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31/10/2023,Macro_ProducerPrices,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2. Lãi suất điều hành SBV

https://www.sbv.gov.vn/webcenter/portal/vi/menu/rm/ls?_afrLoop=34719139833966466#%40%3F_afrLoop%3D34719139833966466%26centerWidth%3D80%2525%26leftWidth%3D20%2525%26rightWidth%3D0%2525%26showFooter%3Dfalse%26showHeader%3Dfalse%26_adf.ctrl-state%3Db21pu33qu_53

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [45]:
# Define the column names as a list
columns = ['MMYYYY','Bank_name', 'CASA_int', 'Off_1M', 'Off_3M', 'Off_6M', 'Off_1Y', 'Off_2Y', 'Onl_1M', 'Onl_3M', 'Onl_6M','Onl_1Y','Onl_2Y']
Update_inst = pd.DataFrame(columns=columns)

In [11]:

SBV_int_url = 'https://www.sbv.gov.vn/webcenter/portal/vi/menu/rm/ls?_afrLoop=34719139833966466#%40%3F_afrLoop%3D34719139833966466%26centerWidth%3D80%2525%26leftWidth%3D20%2525%26rightWidth%3D0%2525%26showFooter%3Dfalse%26showHeader%3Dfalse%26_adf.ctrl-state%3Db21pu33qu_53'
response = requests.get(SBV_int_url)
allsoup = BeautifulSoup(response.text, 'html') 
# Find all rows within the tbody
rows = allsoup.find_all('tr')

# Initialize a list to store the scraped data
data = []

# Refinancing rate
for row in rows:
    cells = row.find_all('td')
    if cells:
        row_data = [cell.get_text(strip=True) for cell in cells]
        data.append(row_data)

InterBank_tb = pd.DataFrame(data[14:21])
Refinancing_tb = pd.DataFrame(data[6:8])

Update_inst.loc[0, 'Bank_name'] = 'SVB_InterBank'
Update_inst.loc[0, "CASA_int"] = InterBank_tb.iloc[0,1]
Update_inst.loc[0, "Off_1M"] = InterBank_tb.iloc[3,1]
Update_inst.loc[0, "Off_3M"] = InterBank_tb.iloc[4,1]
Update_inst.loc[0, "Off_6M"] = InterBank_tb.iloc[5,1]

issue_date = data[12]
pattern = r'\d{1,2}/\d{1,2}/\d{4}'
matches = re.search(pattern, issue_date[0])
if matches:
    date = matches.group()
    Update_inst.loc[0, 'MMYYYY'] = date
else:
    Update_inst.loc[0, 'MMYYYY'] = ''

Update_inst.loc[1, 'Bank_name'] = 'SVB_DiscountRate'
Update_inst.loc[1, 'CASA_int'] = float(Refinancing_tb.iloc[0,1].replace(',', '.').replace('%', ''))
Update_inst.loc[1, 'MMYYYY'] = Refinancing_tb.iloc[0,3]
Update_inst.loc[2, 'Bank_name'] = 'SVB_RefinancingRate'
Update_inst.loc[2, 'CASA_int'] = float(Refinancing_tb.iloc[1,1].replace(',', '.').replace('%', ''))
Update_inst.loc[2, 'MMYYYY'] = Refinancing_tb.iloc[1,3]

In [10]:
Refinancing_tb

,0,1,2,3
0,Lãi suất tái chiết khấu,"3,000%",1123/QĐ-NHNN ngày 16/06/2023,19-06-2023
1,Lãi suất tái cấp vốn,"4,500%",1123/QĐ-NHNN ngày 16/06/2023,19-06-2023


In [12]:
Update_inst

,MMYYYY,Bank_name,CASA_int,Off_1M,Off_3M,Off_6M,Off_1Y,Off_2Y,Onl_1M,Onl_3M,Onl_6M,Onl_1Y,Onl_2Y
0,27/10/2023,SVB_InterBank,"1,37","2,22","3,96","5,22",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,SVB_DiscountRate,"3,000%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,SVB_RefinancingRate,"4,500%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


3. Quyét dữ liệu lịch sử Lãi suất InterBank của SBV

In [2]:
import warnings
import pandas as pd

# Suppress the openpyxl warning
warnings.filterwarnings("ignore")


theframe = pd.DataFrame(columns=['update_date','Qua đêm','1 Tuần','2 Tuần','1 Tháng','3 Tháng','6 Tháng','9 Tháng'])

for x in range (0,25):
    df = pd.read_excel(f"/Users/nguyenhien/Desktop/Interest rate/Macro Economics/LAI_SUAT_TTLNH ({x}).xlsx")
    new_line = [df.iloc[1,1],df.iloc[5,5],df.iloc[6,5],df.iloc[7,5],df.iloc[8,5],df.iloc[9,5],df.iloc[10,5],df.iloc[11,5]]
    new_row = pd.Series(new_line, index=theframe.columns)
    theframe = pd.concat([theframe, new_row.to_frame().T], ignore_index=True)

warnings.filterwarnings("default")


In [3]:
theframe.to_excel("/Users/nguyenhien/Desktop/Interest rate/Macro Economics/LAI_SUAT_TTLNH_Tổng hợp.xlsx")

4. Quyét dữ liệu lịch sử Lãi suất Refinance của SBV

In [ ]:
df

In [17]:
import warnings
import pandas as pd

# Suppress the openpyxl warning
warnings.filterwarnings("ignore")


theframe = pd.DataFrame(columns=['update_date','1','2'])

df_2 = pd.read_excel("C:/Users/hiennpd3/Desktop/Material/macroe/BANG_LAI_SUAT (2).xlsx")
new_line = [df_2.iloc[0,1],df_2.iloc[5,1],df_2.iloc[6,1]]
new_row = pd.Series(new_line, index=theframe.columns)
theframe = pd.concat([theframe, new_row.to_frame().T], ignore_index=True)
theframe


,update_date,1,2
0,3/4/2023,3.5,5.5


In [29]:
for x in range (0,10):
    df = pd.read_excel(f"C:/Users/hiennpd3/Desktop/Material/macroe/BANG_LAI_SUAT ({x}).xlsx")
    if df.iloc[6,1] is None:
        None
    else:
        new_line = [df.iloc[0,1],df.iloc[5,1],df.iloc[6,1]]
        new_row = pd.Series(new_line, index=theframe.columns)
        theframe = pd.concat([theframe, new_row.to_frame().T], ignore_index=True)

# warnings.filterwarnings("default")
theframe.to_excel("C:/Users/hiennpd3/Desktop/Material/macroe/BANG_LAI_SUAT_TH.xlsx")

5. Testing areas

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import datetime 
from decimal import Decimal

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [5]:
# Tạo bảng
columns = ['MMYYYY','Bank_name', 'Bank_code', 'Peer_No', 'Volume_type', 'CASA_int', \
           'Off_1M', 'Off_2M', 'Off_3M', 'Off_6M', \
            'Off_12M', 'Off_13M', 'Off_18M', 'Off_24M', 'Off_36M',\
            'Onl_1M', 'Onl_2M', 'Onl_3M', 'Onl_6M',\
            'Onl_12M', 'Onl_13M', 'Onl_18M','Onl_24M','Onl_36M']


Update_inst = pd.DataFrame(columns=columns)

url_chrome = r"D:\Chromedriver\chromedriver.exe"


In [6]:
Update_inst

,MMYYYY,Bank_name,Bank_code,Peer_No,Volume_type,CASA_int,Off_1M,Off_2M,Off_3M,Off_6M,...,Off_36M,Onl_1M,Onl_2M,Onl_3M,Onl_6M,Onl_12M,Onl_13M,Onl_18M,Onl_24M,Onl_36M


In [7]:
def convert_to_float(value):
    return round(Decimal(float(value.replace(',', '.').replace('%', '').replace('*', ''))),3)

# # Send an HTTP GET request to the URL and retrieve the content
# chrome_options = Options()
# driver = webdriver.Chrome(service=Service(executable_path=url_chrome\
#                                           ), options=chrome_options)
# driver.maximize_window()
# acb = driver.get(url_ACB)


In [18]:
url_ACB = 'https://www.acb.com.vn/lai-suat-tien-gui' # Done

# Send an HTTP GET request to the URL and retrieve the content
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
acb = driver.get(url_ACB)

# Set the common values for 'MMYYYY' and 'Bank_name'
Update_inst.loc[6, 'MMYYYY'] = datetime.datetime.now().strftime("%d/%m/%Y")
Update_inst.loc[6, 'Bank_name'] = 'ACB'


driver.implicitly_wait(20) # driver timeout to stop forever loading

#get page html content
ACB_html_content = driver.page_source
    
# Parse the HTML content using BeautifulSoup
ACB_soup = BeautifulSoup(ACB_html_content, 'html.parser')

# Find and convert offline and online values to float using list comprehension
offline_values_1M =convert_to_float(ACB_soup.find_all("tbody")[1]\
                                   .find_all("tr")[5]\
                                   .find_all("td", class_="xl68")[0]\
                                   .text.strip())
offline_values = [convert_to_float(ACB_soup.find_all("tbody")[1]\
                                   .find_all("tr")[x]\
                                   .find_all("td")[1]\
                                   .text.strip()) for x in [6, 7, 10, 12, 13, 15, 16, 17]]
online_values = [convert_to_float(ACB_soup.find_all("tbody")[3]\
                                  .find_all("tr")[2]\
                                  .find_all("td")[y]\
                                  .text.strip()) for y in [2, 4, 5, 6, 6]]
surplus_values = [convert_to_float(ACB_soup.find_all("tbody")[2]\
                                  .find_all("tr")[1]\
                                  .find_all("td")[x]\
                                  .text.strip()) for x in [1,2,3]]

# Update the DataFrame with the converted values
Update_inst.loc[1, ["Volume_type"]] = 'U300M'
Update_inst.loc[1, ["Off_1M"]] = offline_values_1M + surplus_values[0]
Update_inst.loc[1, ["Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M"]] \
                    = [x + surplus_values[0] for x in offline_values]
Update_inst.loc[1, ["Onl_1M","Onl_3M", "Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]] \
                    = [x + surplus_values[0] for x in online_values]

Update_inst.loc[2, ["Volume_type"]] = 'U10B'
Update_inst.loc[2, ["Off_1M"]] = offline_values_1M + surplus_values[1]
Update_inst.loc[2, ["Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M"]] \
                    = [x + surplus_values[1] for x in offline_values[1]]
Update_inst.loc[2, ["Onl_1M","Onl_3M", "Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]] \
                    = [x + surplus_values[1] for x in online_values[1]]

Update_inst.loc[3, ["Volume_type"]] = '10B'
Update_inst.loc[3, ["Off_1M"]] = offline_values_1M + surplus_values[2]
Update_inst.loc[3, ["Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M"]] \
                    = [x + surplus_values[1] for x in offline_values[2]]
Update_inst.loc[3, ["Onl_1M","Onl_3M", "Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]] \
                    = [x + surplus_values[1] for x in online_values[2]]

# #close driver
# driver.close()
# print(x)


ValueError: could not broadcast input array from shape (5,) into shape (8,)

In [17]:
Update_inst

,MMYYYY,Bank_name,Bank_code,Peer_No,Volume_type,CASA_int,Off_1M,Off_2M,Off_3M,Off_6M,...,Off_36M,Onl_1M,Onl_2M,Onl_3M,Onl_6M,Onl_12M,Onl_13M,Onl_18M,Onl_24M,Onl_36M
6,13/11/2023,ACB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,U300M,NaN,3.300,3.400,3.500,4.600,...,4.700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,U10B,NaN,3.350,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
from decimal import Decimal
print(round(Decimal(surplus_values[0]),2))
# print(round(Decimal(offline_values[3]),2))
# print(round(Decimal(4.6 + 0.1),5))
# print(4.6+0.2)
print(convert_to_float('4.6'))
print(4.6 + 0.1)

0.10
4.600
4.699999999999999


In [8]:
Update_inst

,MMYYYY,Bank_name,Bank_code,Peer_No,Volume_type,CASA_int,Off_1M,Off_2M,Off_3M,Off_6M,...,Onl_6M,Onl_12M,Onl_13M,Onl_18M,Onl_24M,Onl_36M,Off_1Y,Off_2Y,Onl_1Y,Onl_2Y
6,06/11/2023,ACB,NaN,NaN,NaN,NaN,3.3,NaN,3.5,4.8,...,5.0,NaN,NaN,NaN,NaN,NaN,5.3,5.4,5.1,5.1
0,NaN,NaN,NaN,NaN,NaN,0.05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
